In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.metrics import categorical_crossentropy # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [ ]:
MODEL_PATH = "cnn.keras"
PATH_SEPARATOR = "\\"

In [ ]:
# physical_devices = tf.config.experimental.list_physical_devices("GPU")
# print("Num GPUs Available: ", len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Data

In [ ]:
os.getcwd()

In [ ]:
os.chdir("../../datasets/dogs-vs-cats")


if os.getcwd().split(PATH_SEPARATOR)[-1] == "dogs-vs-cats":
    if not os.path.isdir("train/dog"):
        os.makedirs("train/dog", exist_ok=True)
        os.makedirs("train/cat", exist_ok=True)
        os.makedirs("valid/dog", exist_ok=True)
        os.makedirs("valid/cat", exist_ok=True)
        os.makedirs("test/dog", exist_ok=True)
        os.makedirs("test/cat", exist_ok=True)

    if len(os.listdir("test/cat")) == 0:
        for c in random.sample(glob.glob("dog*"), 500):
            shutil.move(c, "train/dog")
        for c in random.sample(glob.glob("cat*"), 500):
            shutil.move(c, "train/cat")
        for c in random.sample(glob.glob("dog*"), 100):
            shutil.move(c, "valid/dog")
        for c in random.sample(glob.glob("cat*"), 100):
            shutil.move(c, "valid/cat")
        for c in random.sample(glob.glob("dog*"), 50):
            shutil.move(c, "test/dog")
        for c in random.sample(glob.glob("cat*"), 50):
            shutil.move(c, "test/cat")

        os.makedirs("data", exist_ok=True)
        excluded_dirs = ["data", "train", "valid", "test"]
        remaining_images = [item for item in os.listdir() if item not in excluded_dirs]
        for c in remaining_images:
            shutil.move(c, "data")

    os.chdir("../../")

In [ ]:
TRAIN_PATH = "datasets/dogs-vs-cats/train"
VALID_PATH = "datasets/dogs-vs-cats/valid"
TEST_PATH = "datasets/dogs-vs-cats/test"

In [ ]:
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=TRAIN_PATH, target_size=(224, 224), classes=["cat", "dog"], batch_size=10
)

valid_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=VALID_PATH, target_size=(224, 224), classes=["cat", "dog"], batch_size=10
)

test_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
).flow_from_directory(
    directory=TEST_PATH,
    target_size=(224, 224),
    classes=["cat", "dog"],
    batch_size=10,
    shuffle=False,
)

In [ ]:
assert train_batches.n == 1000
assert valid_batches.n == 200
assert test_batches.n == 100
assert (
    train_batches.num_classes
    == valid_batches.num_classes
    == test_batches.num_classes
    == 2
)

In [ ]:
def plot_images(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
imgs, labels = next(train_batches)
plot_images(imgs)
print(labels)

# CNN

In [ ]:
model = Sequential(
    [
        Conv2D(
            filters=32,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            input_shape=(224, 224, 3),
        ),
        MaxPool2D(pool_size=(2, 2), strides=2),
        # Dropout(0.2),
        Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding="same"),
        MaxPool2D(pool_size=(2, 2), strides=2),
        # Dropout(0.2),
        Flatten(),
        Dense(2, activation="softmax"),
    ]
)

In [ ]:
model.summary()
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
model.fit(train_batches, validation_data=valid_batches, epochs=10, verbose=2)

# Prediction

In [ ]:
test_imgs, test_labels = next(test_batches)
plot_images(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(test_batches, verbose=0)
rounded_predictions = np.round(predictions)
rounded_predictions

In [ ]:
def plot_confusion_matrix(
    cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")

    print(cm)

    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
cm = confusion_matrix(
    y_true=test_batches.classes, y_pred=np.argmax(rounded_predictions, axis=-1)
)

In [ ]:
test_batches.class_indices

In [ ]:
plot_confusion_matrix(cm=cm, classes=["cat", "dog"])

# Save Model

In [ ]:
try:
    if os.getcwd().split(PATH_SEPARATOR)[-1] != "keras_freeCodeCamp":
        os.chdir("_tutorials/keras_freeCodeCamp")
    model.save(MODEL_PATH)
except Exception as e:
    print(e)
    print("Current working directory is: ", os.getcwd())
    print("Model not saved.")

# Load Model

In [ ]:
try:
    if os.getcwd().split(PATH_SEPARATOR)[-1] != "keras_freeCodeCamp":
        os.chdir("_tutorials/keras_freeCodeCamp")
    model = keras.models.load_model(MODEL_PATH)
    model.summary()
except Exception as e:
    print(e)
    print("Current working directory is: ", os.getcwd())
    print("Model not loaded.")